Data Set Information:

This database contains 76 attributes, but all published experiments refer to using a subset of 14 of them. In particular, the Cleveland database is the only one that has been used by ML researchers to this date. The "goal" field refers to the presence of heart disease in the patient. It is integer valued from 0 (no presence) to 4. Experiments with the Cleveland database have concentrated on simply attempting to distinguish presence (values 1,2,3,4) from absence (value 0).

The names and social security numbers of the patients were recently removed from the database, replaced with dummy values.

One file has been "processed", that one containing the Cleveland database. All four unprocessed files also exist in this directory.

To see Test Costs (donated by Peter Turney), please see the folder "Costs"

Attribute Information:

Only 14 attributes used: 1. #3 (age) 2. #4 (sex) 3. #9 (cp) 4. #10 (trestbps) 5. #12 (chol) 6. #16 (fbs) 7. #19 (restecg) 8. #32 (thalach) 9. #38 (exang) 10. #40 (oldpeak) 11. #41 (slope) 12. #44 (ca) 13. #51 (thal) 14. #58 (num) (the predicted attribute)

Columns:
    age:age in years
    sex:(1 = male; 0 = female)
    cp:chest pain type
    trestbps:resting blood pressure (in mm Hg on admission to the hospital)
    chol:serum cholestoral in mg/dl
    fbs:(fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)
    restecg:resting electrocardiographic results
    thalach:maximum heart rate achieved
    exang:exercise induced angina (1 = yes; 0 = no)
    oldpeak:ST depression induced by exercise relative to rest
    slope:the slope of the peak exercise ST segment
    ca:number of major vessels (0-3) colored by flourosopy
    thal : 3 = normal; 6 = fixed defect; 7 = reversable defect
    target:1 or 0 


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np
import pandas as pd
import pandas_profiling as pp
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Loading the data into dataframe

In [ ]:
df = pd.read_csv("../input/heart-disease-uci/heart.csv")

Viewing the data

In [ ]:
#Once the data is loaded,we can view the data. Instead of viewing the entire data , we can view the first five rows
df.head()

In [ ]:
# We can view the last five rows of the data
df.tail()

Viewing the size of the data (no of rows and no of columns)

In [ ]:
#we can view the total number of rows and columns of data
df.shape

Understanding the column attributes - datatype , no of non null rows

In [ ]:
# Now we got the information about the total size of the data . We can further explore the detailed information about each column using the info method on datadrame
df.info()

The ProfileReport() method gives detailed data summary statistics 

In [ ]:
profile = pp.ProfileReport(df)
profile

In [ ]:
# We can get the summary statistics (min,max,count...) of each column by using the describe() method
df.describe()

In [ ]:
df.corr()

In [ ]:
#Checking the column names
df.columns

In [ ]:
# Rename the columns of dataframe to more meaningful
df=df.rename(columns={'age':'Age','sex':'Sex','cp':'Cp','trestbps':'Trestbps','chol':'Chol','fbs':'Fbs','restecg':'Restecg','thalach':'Thalach','exang':'Exang','oldpeak':'Oldpeak','slope':'Slope','ca':'Ca','thal':'Thal','target':'Target'})

In [ ]:
#Recheck the column names
df.columns

In [ ]:
# Check for missing values
#df.isnull().sum()
df.isnull().mean()

Detecting the Outliers

In [ ]:
# Calculating the IQR for entire dataset, to detect outliers
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

In [ ]:
# Filtering the columns by removing the outliers
print((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))  )

In [ ]:
# Try to delete the rows with outliers and check if this impacts our prediction
df_out = df[(df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))] #viewing the outliers
print(df_out)
#df_out = df[~((df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))).any(axis=1)]

In [ ]:
#df_out.shape # We can see that by deleting the rows with outliers , we may lose a large amount of data

In [ ]:
# We will now try perform imputation on these outliers , As all columns are numerical we can perform median imputation
#df.out = df[(df < (Q1 - 1.5 * IQR)) | (df > (Q3 + 1.5 * IQR))]

Identifying the duplicated values

In [ ]:
# We will try to check and drop duplicate values
#Incase of duplicate rows we use drop_duplicates() method
df_dup = df[df.duplicated()]
df_dup

In [ ]:
#As we have one duplicate row , Delete the duplicated rows
df = df.drop_duplicates()
df

In [ ]:
df['Target'].value_counts()

In [ ]:
sns.countplot(df['Target'])
plt.show()

In [ ]:
df.hist()

Converting the categorical variables

In [ ]:
dataset = pd.get_dummies(df, columns = ['Sex', 'Cp', 'Fbs', 'Restecg', 'Exang', 'Slope', 'Ca', 'Thal'])

Performing Feature Scaling

In [ ]:
standardScaler = StandardScaler()
columns_to_scale = ['Age', 'Trestbps', 'Chol', 'Thalach', 'Oldpeak']
df[columns_to_scale] = standardScaler.fit_transform(df[columns_to_scale])

In [ ]:
df.head()

In [ ]:
y = df['Target']
X = df.drop(['Target'], axis = 1)

Logistic Regression

In [ ]:
logreg = LogisticRegression()
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.4,random_state = 42)
logreg.fit(X_train,y_train)
y_pred = logreg.predict(X_test)
print("The Score is",logreg.score(X_test,y_test))
cv_score_five = cross_val_score(logreg,X,y,cv=5)
cv_score_ten = cross_val_score(logreg,X,y,cv=10)
print("Cross validation score - Five Folds",cv_score_five)
print("Cross validation score - Ten Folds",cv_score_ten)
print("Mean cross validation score",cv_score_ten.mean())
print("Confusion Matrix",confusion_matrix(y_test,y_pred))
print("Classification Report")
print(classification_report(y_test,y_pred))

Decision Tree Classfier

In [ ]:
dec_clf = DecisionTreeClassifier(max_depth = 3 , random_state=1)
dec_clf.fit(X_train,y_train)
y_pred = dec_clf.predict(X_test)
print("The accuarcy score of decision tree classifier is ",accuracy_score(y_test,y_pred))
cv_dec_tree_clf = cross_val_score(dec_clf,X,y,cv=10)
print("The Cross validation score ",cv_dec_tree_clf.mean())

Random Forest Classifier

In [ ]:
randomforest_classifier= RandomForestClassifier(n_estimators=10)
randomforest_classifier.fit(X_train,y_train.ravel())
y_pred = randomforest_classifier.predict(X_test)
cv_score=cross_val_score(randomforest_classifier,X,y,cv=10)
print("The accuarcy score of random tree classifier is ",accuracy_score(y_test,y_pred))
print("The mean cross validation score is ",cv_score.mean())

In [ ]:


## Hyper Parameter Optimization

params={
 "learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ]
    
}



In [ ]:
## Hyperparameter optimization using RandomizedSearchCV
from sklearn.model_selection import RandomizedSearchCV
import xgboost

In [ ]:
classifier=xgboost.XGBClassifier()

In [ ]:
random_search=RandomizedSearchCV(classifier,param_distributions=params,n_iter=5,scoring='roc_auc',n_jobs=-1,cv=5,verbose=3)

In [ ]:
random_search.fit(X,y.ravel())
random_search.best_estimator_



In [ ]:
classifier = xgboost.XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3, gamma=0.3,
              learning_rate=0.2, max_delta_step=0, max_depth=15,
              min_child_weight=5, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
score=cross_val_score(classifier,X,y.ravel(),cv=10)
score

In [ ]:
score.mean()